# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
csvfile = "../Resources/cities.csv"
cities_df = pd.read_csv(csvfile)
cities_df


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

#create locations
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df['Humidity']

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'}

gmaps.figure(layout=figure_layout)
    
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#filter out cities that do not have a max temp bt 77 and 87 and wind speed 10 or less
filter = cities_df.loc[(cities_df['Max Temp'] > 87) | (cities_df['Max Temp'] < 77) | (cities_df['Wind Speed'] > 10) ]
hotel_df = cities_df.drop(filter.index)

#drop useless column 
hotel_df.drop(['Unnamed: 0'], axis= 1, inplace= True)

#display
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df['Hotel Name'] = ' '
hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "prominence",
    "type": "lodging",
    "radius": 5000,
    "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get city name from DF
    city = row['Name']
    # add  to params dict
    params["location"]= f"{row['Latitude']},{row['Longitude']}"
    # assemble url and make API request
    print("Retrieving Results for city " + row['Name'])
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 'NA'

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(hotel_layer)
# Display figure
fig